# 2. Incoming Event Handler
  --------------------------------------------------------------------

Handle incoming events and write them to an output V3IO Stream `incoming-events-stream`.
The received data is partitioned by `user_id`.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

The rest of the notebooks rely on the output stream of this notebook. Therefore, one can change the input data without affecting the rest of the workflow.

## Initialize

Load the project

In [1]:
from mlrun import load_project
from os import path

project_path = path.abspath('conf')
project = load_project(project_path)

Get the generated stream path, this is the input we use and we output to the "incoming events" which is later consumed by other functions

In [2]:
input_stream = project.params.get('STREAM_CONFIGS').get('generated-stream')
input_stream_path =  input_stream.get('path')

Nuclio leverages consumer groups. When one or more Nuclio replicas join a consumer group, each replica receives its equal share of the shards, based on the number of replicas that are defined in the function.

We set up the input stream URL below. A consumer-group URL is in the form of `http://v3io-webapi:8081/<container name>/<stream path>@<consumer group name>`. In this case we use `WEB_API_USERS` for URL prefix `http://v3io-webapi:8081/<container name>` and a consumer group named **`incomingeventhandler`**.

For more information, refer to the [Nuclio v3iostream trigger reference documentation](https://nuclio.io/docs/latest/reference/triggers/v3iostream/).

In [3]:
WEB_API_USERS = project.params.get('WEB_API_USERS')
input_stream_url = path.join(WEB_API_USERS, input_stream_path) + "@incomingeventhandler"
print(f'Input stream URL: {input_stream_url}')

Input stream URL: http://v3io-webapi:8081/users/iguazio/examples/model-deployment-with-streaming/data/generated-stream@incomingeventhandler


Get the incoming-events stream path, this is where we output the data

In [4]:
output_stream = project.params.get('STREAM_CONFIGS').get('incoming-events-stream')
output_stream_path =  output_stream.get('path')
print(f'Output stream path: {output_stream_path}')

Output stream path: iguazio/examples/model-deployment-with-streaming/data/incoming-events-stream


## Create and Test a Local Function 
Import nuclio SDK and magics

In [5]:
import nuclio

In [6]:
# nuclio: start-code

#### Functions imports

In [7]:
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [8]:
%nuclio cmd -c pip install v3io

In [9]:
%%nuclio config
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


## Function code

In [10]:
import os
import json
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    container = os.getenv('CONTAINER')
    output_stream_path = os.getenv('OUTPUT_STREAM_PATH')
    partition_attr = os.getenv('PARTITION_ATTR')
    WEB_API = os.getenv('WEB_API')
    v3io_client = v3io.dataplane.Client(endpoint=WEB_API, access_key=V3IO_ACCESS_KEY)

    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'partition_attr', partition_attr)
    setattr(context, 'container', container)
    setattr(context, 'output_stream_path', output_stream_path)


def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
        
    partition_key = event_dict.get(context.partition_attr)
    record = event_to_record(event_dict, partition_key)
    
    print("saasasas" + context.output_stream_path)
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp.status_code


def event_to_record(event_dict, partition_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [11]:
# nuclio: end-code
# marks the end of a code section

In [12]:
envs = {'V3IO_ACCESS_KEY': os.getenv('V3IO_ACCESS_KEY'),
        'WEB_API' : project.params.get('WEB_API'),
        'CONTAINER': project.params.get('CONTAINER'),
        'OUTPUT_STREAM_PATH': output_stream_path,
        'PARTITION_ATTR': project.params.get('PARTITION_ATTR')}

## Test locally

In [13]:
event = nuclio.Event(body=b'{"user_id" : 111111 , "event_type": "spin"}')
for key, value in envs.items():
    os.environ[key] = str(value)
init_context(context)
handler(context, event)

Python> 2020-08-19 18:49:39,589 [info] Got invoked: {'trigger_kind': '', 'event_body': {'user_id': 111111, 'event_type': 'spin'}}
saasasasiguazio/examples/model-deployment-with-streaming/data/incoming-events-stream
Python> 2020-08-19 18:49:39,591 [info] Sent event to stream: {'record': {'data': '{"user_id": 111111, "event_type": "spin"}', 'partition_key': '111111'}, 'response_status': 200, 'response_body': '{ "FailedRecordCount":0,"Records": [{ "SequenceNumber":1,"ShardId":5 } ] }'}


200

# MLRun

In [14]:
from mlrun import code_to_function

gen_func = code_to_function(name='incoming', kind = 'nuclio')
project.set_function(gen_func)
incoming_event_handler = project.func('incoming')
incoming_event_handler.set_envs(envs)
incoming_event_handler.add_trigger('incoming', nuclio.triggers.V3IOStreamTrigger(url=input_stream_url, access_key=os.getenv('V3IO_ACCESS_KEY'), maxWorkers=10))

In [15]:
project.save()

In [16]:
#Build image
incoming_event_handler.deploy()

> 2020-08-19 18:49:40,978 [info] deploy started
[nuclio] 2020-08-19 18:49:42,050 (info) Build complete
[nuclio] 2020-08-19 18:49:46,085 (info) Function deploy complete
[nuclio] 2020-08-19 18:49:46,091 done updating model-deployment-with-streaming-iguazio-incoming, function address: 3.131.87.251:30342


'http://3.131.87.251:30342'